# Si446x Device Direct Access Tag Position Tracker

In [ ]:
from __future__ import print_function
from builtins import *                  # python3 types
from time import sleep
from datetime import datetime
import struct as pystruct
from binascii import hexlify
import os.path

In [ ]:
!pwd
%autosave 0
import sys
sys.path.append("../si446x/si446x")
%run '../si446x/si446x/notebooks/si446x_Device_Layer.ipynb'

In [ ]:
import sys
sys.path.append("../tagnet/tagnet")
from tagmessages import TagMessage, TagPoll, TagGet, TagPut, TagDelete, TagHead
from tagnames import TagName
from tagtlv import TagTlv, TagTlvList, tlv_types

In [ ]:
import datetime
print('Test Start Time: {}'.format(datetime.datetime.now()))
print('Si446x Radio Device Driver Version: {}'.format(si446x_device_version()))

##  Start up Radio

In [ ]:
radio = si446x_device_start_radio()

In [ ]:
si446x_device_show_config(radio.dump_radio())

## Check for Command Error

In [ ]:
status = radio.get_chip_status()
if (status.chip_pend.CMD_ERROR):
    print(status)

##  Configure Radio

In [ ]:
config = si446x_device_config_radio(radio)

si446x_device_show_config(radio.dump_radio())
total = 0
print('\n=== const config strings:')
for s in config:
    print((hexlify(s)))
    total += len(s) - 4
print('\n total: {}'.format(total))

## Dblk Data Transfer

In [ ]:
from pyproj import Proj, transform
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets.embed import embed_minimal_html
import ipywidgets as widgets

### Repeat fetching Dblk contents until end of file

In [ ]:
# default paramters
MAX_WAIT            = 10
MAX_RECV            = 255
MAX_PAYLOAD         = 254
MAX_RETRIES         = 10
RADIO_POWER         = 100
SHORT_DELAY         = 0
from datetime import datetime

In [ ]:
#"tag"  "sd"    <node_id>   "0"   "dblk"   "0"
def get_dblk_bytes(file_offset, amount_to_get):
    dblk_bytes_name = TagName ('/tag/sd')
    dblk_bytes_name += [TagTlv(tlv_types.NODE_ID, -1),
                        TagTlv(0),
                        TagTlv('dblk'),
                        TagTlv(0),
                        TagTlv(tlv_types.OFFSET, file_offset),
                        TagTlv(tlv_types.SIZE, amount_to_get),
                       ]
    get_dblk = TagGet(dblk_bytes_name)
    print(get_dblk.name)
    req_msg = get_dblk.build()
    si446x_device_send_msg(radio, req_msg, RADIO_POWER);
    rsp_msg, rssi, status = si446x_device_receive_msg(radio, MAX_RECV, 5)
    if(rsp_msg):
        print(hexlify(rsp_msg))
        rsp_obj = TagMessage(rsp_msg)
#        print(rsp_obj.header.options.param.error_code)
#        print(rsp_obj.payload)
        if (rsp_obj.payload):
            pass
        else:
            print("{}".format(rsp_obj.header.options.param.error_code))
#    else:
#        print('TIMEOUT')
    return rsp_obj

In [ ]:
msg = get_dblk_bytes(512,200)

In [ ]:
print(msg.payload[0].value(), msg.payload[1].value(), len(msg.payload[2].value()))
print(hexlify(msg.payload[2].value()))

In [ ]:
start = datetime.now()
for x in range(1000):
    dblk_data = get_dblk_bytes()
    if dblk_data:
        break
    else:
        print("\r{} {}".format(datetime.now() - start, 'timeout'), end="")
        continue
    print("\r{}  {}".format(datetime.now() - start, tag_geo), end="")
    sleep(5)
print("\ntime:{}, lat:{}, lon:{}, elv:{}".format(datetime.now() - start, *tag_geo))

## Extra

In [ ]:
STOP

## Get Chip Status

In [ ]:
print(radio.get_chip_status())

## Get Image Directory

In [ ]:
#"tag"  "sd"    <node_id>   "img"
image_manager_name = TagName ('tag',
                              'sd',
                              TagTlv(tlv_types.NODE_ID, -1),
                              TagTlv(0),
                              TagTlv('img'))
dir_info = TagGet(image_manager_name)
#print(dir_info.name)
dir_msg = dir_info.build()
#print(len(dir_msg),hexlify(dir_msg))
si446x_device_send_msg(radio, dir_msg, RADIO_POWER);
rsp_buf, rssi, status = si446x_device_receive_msg(radio, MAX_RECV, MAX_WAIT)
if (rsp_buf):
#    print(len(rsp_buf),hexlify(rsp_buf))
    rsp_obj = TagMessage(rsp_buf)
    for x in range(0, 8, 2):
        print("state: {}, {}".format(rsp_obj.payload[x+1].value(), rsp_obj.payload[x]))
else:
    print('timeout')

In [ ]:
#"tag"	"sys"	<node_id>	"which"
def get_version(which):
    sys_name = TagName ('tag',
                        'sys',
                        TagTlv(tlv_types.NODE_ID, -1),
                        TagTlv(which))
    sys_obj = TagGet(sys_name)
#    print(sys_obj.name)
    get_msg = sys_obj.build()
    si446x_device_send_msg(radio, get_msg, RADIO_POWER);
    rsp_buf, rssi, status = si446x_device_receive_msg(radio, MAX_RECV, 5)
    if(rsp_buf):
#        print(hexlify(rsp_buf))
        rsp_obj = TagMessage(rsp_buf)
        print("{}: {:^10} state: {}, {}".format(rsp_obj.header.options.param.error_code, which, rsp_obj.payload[1].value(), rsp_obj.payload[0]))

In [ ]:
get_version('active')
get_version('backup')
get_version('golden')
get_version('nib')
get_version('running')

In [ ]:
#"tag"	"sys"	<node_id>	"which"
def set_version(which, version):
    set_name = TagName ('tag',
                        'sys',
                        TagTlv(tlv_types.NODE_ID, -1),
                        TagTlv(which),
                        TagTlv(tlv_types.VERSION, version))
    set_obj = TagPut(set_name)
#    print(set_obj.name)
    set_msg = set_obj.build()
    si446x_device_send_msg(radio, set_msg, RADIO_POWER);
    rsp_buf, rssi, status = si446x_device_receive_msg(radio, MAX_RECV, 5)
    if(rsp_buf):
#        print(hexlify(rsp_buf))
        rsp_obj = TagMessage(rsp_buf)
#        print(rsp_obj.header)
        if (rsp_obj.payload):
            print("{}: state: {}, {}".format(rsp_obj.header.options.param.error_code, rsp_obj.payload[1].value(), rsp_obj.payload[0]))
        else:
            print("{}".format(rsp_obj.header.options.param.error_code))

In [ ]:
set_version('active', (118, 16, 0))

In [ ]:
set_version('backup', (32, 16, 0))

In [ ]:
set_version('running', (125, 1, 0))

## Interactive Group Properties

In [ ]:
from si446xdef import *


def si446x_device_group_fetch_and_decode(group):
    gname = radio_config_group_ids.decoding[group]
    g_s = radio_config_groups[radio_config_group_ids.build(gname)]
    gid = radio_config_group_ids.build(gname)
    p = si446x_device_get_property(radio, gname, 0, g_s.sizeof())
    print(g_s, insert_space(p))
    #print(gname, len(p), hexlify(p))
    print(radio_display_structs[g_s](g_s, p))
    return None


def si446x_device_command_fetch_and_decode(cmd):
    cname = radio_status_cmd_ids.decoding[cmd]
    cfunc, cstr = radio_status_commands[radio_config_cmd_ids.build(cname)]
#    print(cname, cmd)
    if (cfunc):
        cmd = cfunc(cname)
        if (cmd):
#            print(cfunc, hexlify(cmd), cstr)
            radio.spi.command(cmd, cstr)
            rsp = radio.spi.response(cstr.sizeof(), cstr.name)
            if (rsp):
#                print(cstr, radio_display_structs[cstr])
                print(cstr, insert_space(rsp))
                print(radio_display_structs[cstr](cstr, rsp))
            else:
                print('no response')
        else:
            print('no command')
    else:
        print('no function')
    return None

In [ ]:
interact(si446x_device_group_fetch_and_decode, group=radio_config_group_ids.encoding)

## Interactive  Command Status Responses

In [ ]:
interact(si446x_device_command_fetch_and_decode, cmd=radio_status_cmd_ids.encoding)

In [ ]:
from datetime import datetime
datetime.now()